In [13]:
import paramiko
import os
import datetime
import pandas as pd
import numpy as np
print(datetime.datetime.now())
os.getcwd()

2020-05-12 13:22:55.354365


'/home/jian/Projects/Big_Lots/Analysis/2020_Q2/vulnerable_competitors'

In [3]:
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'audience_from_SP',
 'copy_from_45_and_upload_to_FB.ipynb']

In [2]:
host = "45.32.4.178"
port = 22
password = "juba-jl1"
username = "jian"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [4]:
list_remote_files=[
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/Levin & Wolf_1201_0315.csv",
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/At Home_1201_0315.csv",
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/Art Van Furniture_1201_0315.csv",
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/Pier 1_1201_0315.csv",
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/Macy's_1201_0315.csv",
    "/storage/sg_outputs/biglots/finaloutputs/output_120119_031520/Bed Bath & Beyond_1201_0315.csv"
]

In [5]:
for remote_file in list_remote_files:
    basename=os.path.basename(remote_file)
    local_path="./audience_from_SP/"+basename
    sftp.get(remote_file,local_path)
sftp.close()
transport.close()

In [7]:
import glob
list_local_files=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2020_Q2/vulnerable_competitors/audience_from_SP/*.csv")

for local_file in list_local_files:
    df=pd.read_csv(local_file,dtype=str,nrows=10)
    print(os.path.basename(local_file),df.columns.tolist())

Levin & Wolf_1201_0315.csv ['ad_id', 'safegraph_place_id']
At Home_1201_0315.csv ['ad_id', 'safegraph_place_id']
Art Van Furniture_1201_0315.csv ['ad_id', 'safegraph_place_id']
Pier 1_1201_0315.csv ['ad_id', 'safegraph_place_id']
Macy's_1201_0315.csv ['ad_id', 'safegraph_place_id']
Bed Bath & Beyond_1201_0315.csv ['ad_id', 'safegraph_place_id']


In [14]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
BigLotsAccount_id="act_271491453638620"

# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBAN7XRyNl2ZCmRZA2vdLIlCsRXcoNagPlZBZAywKS3GtKLMiEl9ZChTwC4w0z6NjQA78uNbLV6FpCnqNkIk4Tptk7ec02gKtLNkSR0ZB0CUcjK5Aq7cQY6O1fkEEDvAx1AGZBQ4rtfjtWmxph9O6Bi5KCSTQKZApDnQZDZD"



def creatfbaudience_from_adid(AudienceName,AudienceDescription,df,ACCESS_TOKEN,accountid,):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    audience = CustomAudience(fbname)
    users = list(df['ad_id'])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(CustomAudience.Schema.mobile_advertiser_id, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)

In [15]:
for file in list_local_files:
    audience_name = os.path.basename(file).split(".")[0]
    description=os.path.basename(file).split(".")[0]+"_uploaded_on_"+str(datetime.datetime.now().date())+"_JL_for_vulnerable competitors"
    df = pd.read_csv(file,dtype=str,usecols=['ad_id'])
    creatfbaudience_from_adid(AudienceName=audience_name,
                    AudienceDescription=description,
                    df=df,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id)

2020-05-12 13:25:02.733758 Levin & Wolf_1201_0315 uploaded 55234
2020-05-12 13:25:26.845217 At Home_1201_0315 uploaded 329437
2020-05-12 13:25:42.167358 Art Van Furniture_1201_0315 uploaded 223272
2020-05-12 13:28:01.843798 Pier 1_1201_0315 uploaded 1904030
2020-05-12 13:30:30.564975 Macy's_1201_0315 uploaded 1934104
2020-05-12 13:32:38.369826 Bed Bath & Beyond_1201_0315 uploaded 1867125
